In [1]:
# ------------------------------------
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT License.
# ------------------------------------

from typing import Optional, Dict, Any
import os
import time
from pathlib import Path
from dotenv import load_dotenv

# Azure imports
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from azure.ai.evaluation.red_team import RedTeam, RiskCategory, AttackStrategy
from azure.ai.agents import AgentsClient
from azure.ai.agents.models import ListSortOrder

load_dotenv()

import asyncio

In [3]:
 # Initialize Azure credentials
credential = DefaultAzureCredential()

# Get AI project parameters from environment variables (matching evaluate.py)
project_endpoint = os.getenv("PROJECT_ENDPOINT")
deployment_name = os.getenv("MODEL_DEPLOYMENT_NAME")  
agent_id = os.getenv("AZURE_EXISTING_AGENT_ID")
agent_name = os.getenv("AZURE_EXISTING_AGENT_NAME")

print(f"Using project endpoint: {project_endpoint}")
print(f"Using deployment name: {deployment_name}")
print(f"Using agent ID: {agent_id}")
print(f"Using agent name: {agent_name}")

Using project endpoint: https://aq-ai-foundry-sweden-central.services.ai.azure.com/api/projects/firstProject
Using deployment name: gpt-4o-mini
Using agent ID: asst_RHh3u68FXNJYxb5MPt7Ysso0
Using agent name: my-agent-with-tracing


In [4]:
# initialize the agents client
# This client will be used to interact with the Azure AI Agents service
project_client = AgentsClient(endpoint=project_endpoint, credential=credential)
print("Project client initialized.")

Project client initialized.


In [5]:
agent = project_client.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name="my-agent",
    instructions="You are a GenAI agent. Always answer in French",
)
print(f"Created agent, agent ID: {agent.id}")

Created agent, agent ID: asst_k5Y1tmn43CJjiimWzkXoMhWP


In [6]:
# Print agent details to verify correct targeting
print(f"We are going to start running Red Team evaluation against recently created agent:")
print(f"  - Agent ID: {agent.id}")
print(f"  - Agent Name: {agent.name}")
print(f"  - Using Model: {deployment_name}")

We are going to start running Red Team evaluation against recently created agent:
  - Agent ID: asst_k5Y1tmn43CJjiimWzkXoMhWP
  - Agent Name: my-agent
  - Using Model: gpt-4o-mini


In [7]:
def agent_callback(query: str) -> str:

    # [START create_thread]
    thread = project_client.threads.create()
    # [END create_thread]
    print(f"Created thread, thread ID: {thread.id}")
    # [START create_message]
    message = project_client.messages.create(thread_id=thread.id, role="user", content=query)
    # [END create_message]
    print(f"Created message, message ID: {message.id}")
    # [START create_run]
    run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
    # [END create_run]
    print(f"Created run, run ID: {run.id}")


    # Poll the run as long as run status is queued or in progress
    while run.status in ["queued", "in_progress", "requires_action"]:
        # Wait for a second
        time.sleep(1)
        run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
        # [END create_run]
        print(f"Run status: {run.status}")

        if run.status == "failed":
            print(f"Run error: {run.last_error}")
            return "Error: Agent run failed."
        messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.DESCENDING)
        for msg in messages:
            if msg.text_messages:
                    return msg.text_messages[0].text.value
        return "Could not get a response from the agent."

In [9]:
# Instantiate your AI Red Teaming Agent
# Specifying risk categories and number of attack objectives per risk categories you want the AI Red Teaming Agent to cover
red_team_agent = RedTeam(
    azure_ai_project=project_endpoint, # required
    credential=DefaultAzureCredential(), # required
    risk_categories=[ # optional, defaults to all four risk categories
        RiskCategory.Violence,
        RiskCategory.HateUnfairness,
        RiskCategory.Sexual,
        RiskCategory.SelfHarm
    ], 
    num_objectives=5, # optional, defaults to 10
)

Class RedTeam: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [ ]:
print("Starting Red Team scan...")
result = await red_team_agent.scan(
    target=agent_callback,
    scan_name="Agent-Scan",
    attack_strategies=[AttackStrategy.Flip],
)
print("Red Team scan complete.")

Starting Red Team scan...
🚀 STARTING RED TEAM SCAN: Agent-Scan
📂 Output directory: ./.scan_Agent-Scan_20250613_091707
📊 Risk categories: ['violence', 'hate_unfairness', 'sexual', 'self_harm']
🔗 Track your red team scan in AI Foundry: https://ai.azure.com/resource/build/redteaming/a4e1fb73-d4c5-4c77-b48e-46177f1ca796?wsid=/subscriptions/7a28b21e-0d3e-4435-a686-d92889d4ee96/resourceGroups/AI-FOUNDRY-RG/providers/Microsoft.CognitiveServices/accounts/aq-ai-foundry-Sweden-Central/projects/firstProject&tid=a172a259-b1c7-4944-b2e1-6d551f954711
📋 Planning 8 total tasks


Scanning:   0%|                         | 0/8 [00:00<?, ?scan/s, current=fetching baseline/violence]

📚 Using attack objectives from Azure RAI service


Scanning:   0%|                  | 0/8 [00:01<?, ?scan/s, current=fetching baseline/hate_unfairness]

📝 Fetched baseline objectives for violence: 5 objectives


...

📝 Fetched baseline objectives for sexual: 5 objectives


Scanning:   0%|                             | 0/8 [00:02<?, ?scan/s, current=fetching flip/violence]

📝 Fetched baseline objectives for self_harm: 5 objectives
🔄 Fetching objectives for strategy 2/2: flip


Scanning:   0%|                                          | 0/8 [00:03<?, ?scan/s, current=batch 1/2]

⚙️ Processing 8 tasks in parallel (max 5 at a time)
▶️ Starting task: baseline strategy for violence risk category
▶️ Starting task: baseline strategy for hate_unfairness risk category
▶️ Starting task: baseline strategy for sexual risk category
▶️ Starting task: baseline strategy for self_harm risk category
▶️ Starting task: flip strategy for violence risk category
Created thread, thread ID: thread_hSyrXWmHUqwEe4Ul0nnywTQ7
Created message, message ID: msg_atPOmuXsMA1BbcRJQrnDMDvZ
Created run, run ID: run_U60jr3DJF2O8MjohKD42S0FQ
Run status: RunStatus.FAILED
Run error: {'code': 'invalid_engine_error', 'message': 'Failed to resolve model info for: gpt-4o-mini'}
Created thread, thread ID: thread_8u1dSDcSxtG77GMP5IqVZiur
Created message, message ID: msg_C2e5gUQaEwyREkaTtU5Ozus1
Created run, run ID: run_t4s1d5urrFBezHyOnyQaQbbD
Run status: RunStatus.FAILED
Run error: {'code': 'invalid_engine_error', 'message': 'Failed to resolve model info for: gpt-4o-mini'}
Created thread, thread ID: thre